In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/FYP/chest_xray_VGG16_Augmentation>.zip -d /content/extracted_files

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Define dataset paths
base_dir = '/content/extracted_files/chest_xray_VGG16_Augmentation'  # Adjust this path as needed
train_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_VGG16_Augmentation/train')
test_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_VGG16_Augmentation/test')


In [ ]:
# Define data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)


In [ ]:
# Create training and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 input size
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 4187 images belonging to 2 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 1045 images belonging to 2 classes.


In [ ]:
# Create test dataset without augmentation
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [ ]:
# Load VGG16 without the top layer
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the convolutional base
for layer in vgg_base.layers:
    layer.trainable = False


In [ ]:
# Build the full model
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,926,209 (68.38 MB)

 Trainable params: 3,211,521 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust epochs as needed
    verbose=1
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 3159s 24s/step - accuracy: 0.7916 - loss: 0.5959 - val_accuracy: 0.9100 - val_loss: 0.1913
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3212s 25s/step - accuracy: 0.9030 - loss: 0.2413 - val_accuracy: 0.9368 - val_loss: 0.1727
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3202s 24s/step - accuracy: 0.9066 - loss: 0.2147 - val_accuracy: 0.9378 - val_loss: 0.1572
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3141s 24s/step - accuracy: 0.9172 - loss: 0.2049 - val_accuracy: 0.9435 - val_loss: 0.1539
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3126s 24s/step - accuracy: 0.9188 - loss: 0.2030 - val_accuracy: 0.9378 - val_loss: 0.1493
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3113s 24s/step - accuracy: 0.9191 - loss: 0.1906 - val_accuracy: 0.9206 - val_loss: 0.1822
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3113s 24s/step - accuracy: 0.9248 - loss: 0.1826 - val_accuracy: 0.9426 - val_loss: 0.1389
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3133s 24s/step - accuracy: 0.9326 - loss: 0.1804 - val

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 361s 18s/step - accuracy: 0.8839 - loss: 0.3397
Test Accuracy: 90.38%


In [ ]:
# Predict classes on test data
predictions = (model.predict(test_generator) > 0.5).astype("int32")


20/20 ━━━━━━━━━━━━━━━━━━━━ 360s 18s/step


In [ ]:
# Generate Confusion Matrix and Classification Report
print("Confusion Matrix")
print(confusion_matrix(test_generator.classes, predictions))


Confusion Matrix
[[196  38]
 [ 22 368]]


In [ ]:
print("Classification Report")
print(classification_report(test_generator.classes, predictions, target_names=test_generator.class_indices.keys()))


Classification Report
              precision    recall  f1-score   support

      NORMAL       0.90      0.84      0.87       234
   PNEUMONIA       0.91      0.94      0.92       390

    accuracy                           0.90       624
   macro avg       0.90      0.89      0.90       624
weighted avg       0.90      0.90      0.90       624

